# Baseline: House Price Prediction

Этот ноутбук строит минимальный рабочий бейзлайн для задачи предсказания цены дома (Kaggle House Prices).

Шаги:
- Загрузка данных
- Препроцессинг (импутация, One-Hot Encoding)
- Бейзлайн-модель (RandomForestRegressor)
- K-Fold CV (RMSE)
- Hold-out оценка
- Сохранение модели


In [ ]:
# Imports and setup
import os
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

PROJECT_ROOT = os.path.abspath(os.path.join(os.path.dirname(".."), ".."))
DATA_DIR = os.path.join(PROJECT_ROOT, "data", "raw")
TRAIN_PATH = os.path.join(DATA_DIR, "train.csv")
MODELS_DIR = os.path.join(PROJECT_ROOT, "models")
os.makedirs(MODELS_DIR, exist_ok=True)

TRAIN_PATH


In [ ]:
# Load data
import pathlib
from IPython.display import display

if not os.path.exists(TRAIN_PATH):
    raise FileNotFoundError(f"Place train.csv into {DATA_DIR}")

df = pd.read_csv(TRAIN_PATH)
display(df.head())
print(df.shape)
assert "SalePrice" in df.columns, "train.csv must contain SalePrice column"


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

print("[ШАГ 3] Построение графиков визуализации...")

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

axes[0, 0].hist(df['SalePrice'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Распределение цен на дома (SalePrice)', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Цена ($)', fontsize=12)
axes[0, 0].set_ylabel('Количество домов', fontsize=12)
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].boxplot(df['SalePrice'])
axes[0, 1].set_title('Boxplot цен на дома', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('Цена ($)', fontsize=12)
axes[0, 1].grid(True, alpha=0.3)

if 'OverallQual' in df.columns:
    axes[1, 0].scatter(df['OverallQual'], df['SalePrice'], alpha=0.5, s=20)
    axes[1, 0].set_title('Качество vs Цена', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('OverallQual (качество)', fontsize=12)
    axes[1, 0].set_ylabel('Цена ($)', fontsize=12)
    axes[1, 0].grid(True, alpha=0.3)

if 'GrLivArea' in df.columns:
    axes[1, 1].scatter(df['GrLivArea'], df['SalePrice'], alpha=0.5, s=20)
    axes[1, 1].set_title('Жилая площадь vs Цена', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('GrLivArea (кв. футы)', fontsize=12)
    axes[1, 1].set_ylabel('Цена ($)', fontsize=12)
    axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("=" * 50)
print("СТАТИСТИКА ПО ЦЕНАМ (SalePrice):")
print("=" * 50)
print(f"Средняя цена: ${df['SalePrice'].mean():,.2f}")
print(f"Медианная цена: ${df['SalePrice'].median():,.2f}")
print(f"Минимальная цена: ${df['SalePrice'].min():,.2f}")
print(f"Максимальная цена: ${df['SalePrice'].max():,.2f}")
print(f"Стандартное отклонение: ${df['SalePrice'].std():,.2f}")
print("=" * 50)


In [ ]:
# Split target and features
y = df["SalePrice"]
X = df.drop(columns=["SalePrice"]) 

num_cols = X.select_dtypes(include=["int64", "float64"]).columns
cat_cols = X.select_dtypes(include=["object"]).columns

len(num_cols), len(cat_cols), list(cat_cols)[:5]


In [ ]:
# Preprocessing and model
numeric_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median"))
])

categorical_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, num_cols),
        ("cat", categorical_pipe, cat_cols),
    ]
)

model = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)

pipe = Pipeline(steps=[
    ("prep", preprocess),
    ("model", model)
])

pipe


In [ ]:
# Cross-validation (RMSE)
import numpy as np
scores = cross_val_score(pipe, X, y, cv=5, scoring="neg_root_mean_squared_error", n_jobs=-1)
rmse_mean, rmse_std = (-scores).mean(), (-scores).std()
rmse_mean, rmse_std


In [ ]:
# Hold-out validation
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
pipe.fit(X_train, y_train)
valid_pred = pipe.predict(X_valid)
holdout_rmse = mean_squared_error(y_valid, valid_pred, squared=False)
holdout_rmse


In [ ]:
# Save model pipeline
model_path = os.path.join(MODELS_DIR, "baseline_random_forest.pkl")
joblib.dump(pipe, model_path)
model_path


In [ ]:
import joblib
pipe = joblib.load("../models/baseline_random_forest.pkl")
pipe


In [ ]:
# Predict house prices for Kaggle test set
TEST_PATH = os.path.join(DATA_DIR, "test.csv")
if not os.path.exists(TEST_PATH):
    raise FileNotFoundError(f"Place test.csv into {DATA_DIR}")

test_df = pd.read_csv(TEST_PATH)
# подключаем обученный пайплайн (из памяти или из файла, если перезапускали ноутбук)
if 'pipe' not in globals():
    pipe = joblib.load(os.path.join(MODELS_DIR, "baseline_random_forest.pkl"))

test_predictions = pipe.predict(test_df)

test_df.assign(SalePrice=test_predictions).loc[:, ["Id", "SalePrice"]].head()
